# Random Compiling
QNLib currently supports multi qudit random compiling which twirls single qubit gates to convert noise channels to Pauli depolarizing noise.

See "Noise tailoring for scalable quantum computation via randomized compiling" by Wallman and Emerson (2016).

Consider an arbitrary circuit, say, a random 3-qutrit Clifford generated by the convenient Tableau class:

In [1]:
from qnlib.benchmarking import Tableau

n=3
d=3

t = Tableau(n, d)
t.populate_and_sweep(display=False)
c = t.circuit
print(c)

                                ┌────┐                   ┌────┐
0 (d=3): ───S───────CX0───CX0────CX0─────CX0───F───CX0────CX0─────CX0───F─────M2────────────────────────────────────────────────
                    │     │      │       │         │      │       │
1 (d=3): ───S───S───CX1───CX1────┼──S────┼─────────CX1────┼──S────┼─────CX1───CX1───F───S───CX1───F───M2───Y(3)───Y(3)──────────
                                 │       │                │       │     │     │             │
2 (d=3): ───F────────────────────CX2─────CX2──────────────CX2─────CX2───CX2───CX2───F───────CX2───F───F────S──────F──────X(3)───
                                └────┘                   └────┘


We twirl the single qubit gates with random Paulis such that the overall logic is the same but the noise is transformed to Pauli noise:

In [2]:
from qnlib.compiling import RandomCompiling
rc = RandomCompiling(c, d)
rc.compile()
ac = rc.arranged_circuit
print(ac)

0 (d=3): ───S───T2,0──────────CX0───CXinvc───T2,0_inv───CX0───T3,0───CX0───CXinvc───T3,0_inv───CX0───T4,0───CX0───CXinvc───T4,0_inv───CX0───T5,0──────────CX0───CXinvc───T5,0_inv───CX0───F───T7,0───CX0───CXinvc───T7,0_inv───CX0───T8,0───CX0───CXinvc───T8,0_inv───CX0───T9,0──────────CX0───CXinvc───T9,0_inv───CX0───F───T10,0─────────T10,0_inv─────────────────────M2───T11,0─────────T11,0_inv─────────────────────────T14,0─────────────────T14,0_inv───────────────────────────────────────────────────
                              │     │                   │            │     │                   │            │     │                   │                   │     │                   │                │     │                   │            │     │                   │                   │     │                   │
1 (d=3): ───S───S──────T2,1───CX1───CXinvt───T2,1_inv───CX1───T3,1───CX1───CXinvt───T3,1_inv───CX1───T4,1───┼─────┼────────T4,1_inv───┼─────S──────T5,1───┼─────┼────────T5,1_inv───┼─────────T7